# Project III: Classical Non-linear Models and Police use of force
This notebook uses the Police Public Contact Survey (PPCS) dataset: `ppcs_cc.csv`.

In [1]:
%load_ext autoreload
%autoreload 2

import pandas as pd 
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt 
import probit as probit
import logit as logit
import estimation as est
from scipy.stats import norm
from scipy.stats import t

### Load the data 

In [2]:
# Load the dataset
dat = pd.read_csv('ppcs_cc.csv')

# Inspect distribution of the target variable
print("\nDistribution of 'anyuseofforce_coded':")
print(dat['anyuseofforce_coded'].value_counts(normalize=True))

# Inspect value counts for categorical variables
categorical_vars = ["sblack", "shisp", "swhite", "sother", "smale", "omajblack", 
                    "omajhisp", "omajwhite", "omajother", "osplit", "inctype_lin", "sbehavior"]

for var in categorical_vars:
    print(f"\nValue Counts for {var}:")
    print(dat[var].value_counts())


Distribution of 'anyuseofforce_coded':
anyuseofforce_coded
0    0.994999
1    0.005001
Name: proportion, dtype: float64

Value Counts for sblack:
sblack
0    3379
1     420
Name: count, dtype: int64

Value Counts for shisp:
shisp
0    3413
1     386
Name: count, dtype: int64

Value Counts for swhite:
swhite
1    2808
0     991
Name: count, dtype: int64

Value Counts for sother:
sother
0    3614
1     185
Name: count, dtype: int64

Value Counts for smale:
smale
1    2012
0    1787
Name: count, dtype: int64

Value Counts for omajblack:
omajblack
0    3568
1     231
Name: count, dtype: int64

Value Counts for omajhisp:
omajhisp
0    3708
1      91
Name: count, dtype: int64

Value Counts for omajwhite:
omajwhite
1    3433
0     366
Name: count, dtype: int64

Value Counts for omajother:
omajother
0    3755
1      44
Name: count, dtype: int64

Value Counts for osplit:
osplit
0    3799
Name: count, dtype: int64

Value Counts for inctype_lin:
inctype_lin
2    3641
1     158
Name: count, dtype

In [3]:
# Declare labels    
y_lab = 'anyuseofforce_coded'
#x_lab = ['const', 'sblack', 'shisp', 'sother']
#x_lab = ['const', 'sblack', 'shisp', 'sother', 'smale', 'sempl', 'sincome', 'spop', 'sage', 'sagesq']
#x_lab = ['const', 'sblack', 'shisp', 'sother', 'smale', 'sempl', 'sincome', 'spop', 'sage', 'sagesq', 'daytime', 'inctype_lin', 'omajblack', 'omajhisp', 'omajother']
x_lab = ['const', 'sblack', 'shisp', 'sother', 'smale', 'sempl', 'sincome', 'spop', 'sage', 'sagesq', 'daytime', 'inctype_lin', 'omajblack', 'omajhisp', 'omajother', 'sbehavior']

dat['sage'] = dat['sage'] / 10
dat['sagesq'] = dat.sage * dat.sage 

#['const', 'sblack', 'shisp', 'sother', 'smale', 'sage', 'sempl', 'sincome', 'spop', 'daytime', 'inctype_lin', 'omajblack', 'omajhisp', 'omajother', 'sbehavior', 'year']  # Dropped 'osplit' and year // Multikollinearitet mellem hhv. "omajblack", "omajhisp" "omajwhite" og "omajother" +  "sblack", "shisp", "swhite", "sother"

## Declare labels, her er sother og omajother taget ud, men I kan bare rette til white
#y_lab = 'anyuseofforce_coded'
#x_lab = ["const", "sblack", "shisp", "swhite",  # Drop 'sother' giver Multikollinearitet  sammen med "sblack", "shisp", "swhite"
#         "smale", "sage", "sempl", "sincome", 
#         "spop", "daytime", "inctype_lin", 
#         "omajblack", "omajhisp", "omajwhite",  # Drop 'omajother' giver Multikollinearitet sammen med "omajblack", "omajhisp", "omajwhite"
#         "sbehavior"] # dropped year and osplit (indeholder kun 0?)

# create extra variables 
N = dat.shape[0]
dat['const'] = np.ones((N,))

# Rebuild the dataset without 'osplit'
dat = dat[[y_lab] + x_lab].copy()

# Check for missing data
assert dat.notnull().all(axis=1).all(), 'Missing values detected. Clean your data!'

dat.tail(5)

,anyuseofforce_coded,const,sblack,shisp,sother,smale,sempl,sincome,spop,sage,sagesq,daytime,inctype_lin,omajblack,omajhisp,omajother,sbehavior
3794,0,1.0,0,0,0,0,1,3,1,7.2,51.84,0,1,0,0,0,1
3795,0,1.0,0,0,0,0,0,2,1,7.1,50.41,1,2,0,0,0,0
3796,0,1.0,0,0,0,0,0,1,1,7.6,57.76,1,2,0,0,0,0
3797,0,1.0,0,0,0,0,0,3,4,7.9,62.41,1,2,0,0,0,0
3798,0,1.0,0,0,0,0,0,2,1,7.5,56.25,1,2,0,0,0,0


In [4]:
# Extract y and X
y = dat[y_lab].values
x = dat[x_lab].values
K = x.shape[1]

In [5]:
count_violent_1 = (dat['anyuseofforce_coded'] == 1).sum()
print(f"Number of 1s in 'anyuseofforce_coded': {count_violent_1}")

Number of 1s in 'anyuseofforce_coded': 19


## Estimate using Probit

In [6]:
# Initialize starting values
theta0 = probit.starting_values(y, x)

# Display starting values
print("Starting values for theta:", theta0)

probit_results = est.estimate(probit.q, theta0, y, x, cov_type='Sandwich')

Starting values for theta: [ 0.13618175  0.00493452  0.02218758 -0.00094007  0.01082135 -0.0155645
  0.00220925  0.01026053  0.00248039 -0.00072741 -0.00447688 -0.07037371
 -0.01365712 -0.00313071 -0.01591562  0.08840108]
Optimization terminated successfully.
         Current function value: 0.022253
         Iterations: 160
         Function evaluations: 3026
         Gradient evaluations: 178


In [7]:
probit_tab = est.print_table(x_lab, probit_results, title=f'Probit, y = {y_lab}')
probit_tab

Optimizer succeded after 160 iter. (3026 func. evals.). Final criterion:  0.02225.
Probit, y = anyuseofforce_coded


,theta,se,t
const,-2.6164,0.6960,-3.7594
sblack,0.2041,0.2996,0.6812
shisp,0.4162,0.2386,1.7443
sother,0.1052,0.4427,0.2376
smale,0.5617,0.2102,2.6730
sempl,-0.5000,0.2206,-2.2659
sincome,0.1022,0.1245,0.8205
spop,0.1988,0.0747,2.6628
sage,0.4862,0.3720,1.3070
sagesq,-0.0791,0.0498,-1.5861


## Estimate using Logit

In [8]:
logit_results = est.estimate(logit.q, theta0, y, x, cov_type='Sandwich')

Optimization terminated successfully.
         Current function value: 0.022389
         Iterations: 201
         Function evaluations: 3604
         Gradient evaluations: 212


In [9]:
logit_tab = est.print_table(x_lab, logit_results, title=f'Logit, y = {y_lab}')
logit_tab

Optimizer succeded after 201 iter. (3604 func. evals.). Final criterion:  0.02239.
Logit, y = anyuseofforce_coded


,theta,se,t
const,-5.6083,1.9346,-2.8990
sblack,0.4374,0.8095,0.5403
shisp,0.9406,0.6140,1.5318
sother,-0.0907,1.2971,-0.0700
smale,1.1538,0.5732,2.0130
sempl,-1.1621,0.6147,-1.8903
sincome,0.2021,0.3417,0.5916
spop,0.4811,0.1904,2.5275
sage,1.3022,1.1764,1.1070
sagesq,-0.2206,0.1652,-1.3353


## Average partial effects

### Probit

In [10]:
# Estimating the average partial effects 
indices = [x_lab.index('sblack'), x_lab.index('shisp'), x_lab.index('sother')]  
labels = ['sblack', 'shispanic', 'sother'] 
probit.properties(x, probit_results['theta'],probit_results['cov'],print_out = True,se=True,indices=indices, labels = labels)

,Estimate,SE,t-value,p-value
sblack,0.003,0.004,0.587,0.557
shispanic,0.006,0.005,1.333,0.183
sother,0.001,0.006,0.216,0.829


### Logit

In [11]:
# Estimating the average partial effects 
indices = [x_lab.index('sblack'), x_lab.index('shisp'), x_lab.index('sother')]  
labels = ['sblack', 'shispanic', 'sother']  
logit.properties(x, logit_results['theta'],logit_results['cov'],print_out = True,se=True,indices=indices, labels = labels)

,Estimate,SE,t-value,p-value
sblack,0.002,0.005,0.471,0.638
shispanic,0.005,0.005,1.176,0.240
sother,-0.000,0.005,-0.072,0.942


## Partial Effects

#### Defining different fixed vectors

In [12]:
#means of the regressors
print(f"{np.mean(dat['sage']):.2f}")
print(f"{np.mean(dat['sagesq']):.2f}")
print(f"{np.mean(dat['sincome']):.2f}")
print(f"{np.mean(dat['spop']):.2f}")

4.10
19.42
2.16
1.36


In [13]:
###ORIGINAL VECTOR###
# Let us make a vector of the values we want to investigate
x_lab = ['const', 'sblack', 'shisp', 'sother', 'smale', 'sage', 'sempl', 'sincome', 'spop', 'daytime', 'inctype_lin', 'omajblack', 'omajhisp', 'omajother', 'sbehavior','sagesq']

x_me = np.array([1, 0, 0, 0, 1, 4.10, 0, 2.16, 1.36,0,1,0,0,0, 0,19.42]).reshape(1, -1)
pd.DataFrame(x_me, columns=x_lab, index=['x_me'])


,const,sblack,shisp,sother,smale,sage,sempl,sincome,spop,daytime,inctype_lin,omajblack,omajhisp,omajother,sbehavior,sagesq
x_me,1.0,0.0,0.0,0.0,1.0,4.1,0.0,2.16,1.36,0.0,1.0,0.0,0.0,0.0,0.0,19.42


In [14]:
### BEHAVIOR = 1 ###
# Let us make a vector of the values we want to investigate
#x_me= np.array([1, 0, 0, 0, 1, 4.1, 0, 2.16, 1.36,0,1,0,0,0,1,19.42]).reshape(1, -1)
#pd.DataFrame(x_me, columns=x_lab, index=['x_behavior'])


In [15]:
###DAYTIME = 1###
# Let us make a vector of the values we want to investigate
#x_me = np.array([1, 0, 0, 0, 1, 4.1, 0, 2.16, 1.36,1,1,0,0,0,0,19.42]).reshape(1, -1)
#pd.DataFrame(x_me, columns=x_lab, index=['x_daytime'])


## Swiching race from white to black, hispanic, other

In [16]:
# We will look at the same values as previously, but we want to look at the difference for foreign = 0 and foreign = 1.
#k=1: black 
#k=2: hispanic
#k=3: other

k = 1
x_me2 = x_me.copy()
x_me2[:, k] = 1  # Keep everythin the same, but change foreign to 1 for all obs. 
pd.DataFrame(x_me2, columns=x_lab, index=['x_me2'])

,const,sblack,shisp,sother,smale,sage,sempl,sincome,spop,daytime,inctype_lin,omajblack,omajhisp,omajother,sbehavior,sagesq
x_me2,1.0,1.0,0.0,0.0,1.0,4.1,0.0,2.16,1.36,0.0,1.0,0.0,0.0,0.0,0.0,19.42


### Probit

In [17]:
b_pr = probit_tab.theta.values

me_black_pr = probit.G(x_me2@b_pr) - probit.G(x_me@b_pr) 

In [18]:
gx0 = norm.pdf(x_me@b_pr)
gx2 = norm.pdf(x_me2@b_pr)

grad_d_pr = gx2*x_me2 - gx0*x_me

In [19]:
def get_se(grad, cov):
    cov_me = grad@cov@grad.T
    return np.sqrt(np.diag(cov_me))

se_d_pr = get_se(grad_d_pr, probit_results['cov'])

In [20]:
me_dict = {'Marginal Effect': me_black_pr[0],
           's.e.':            se_d_pr}
tab = pd.DataFrame(me_dict)
tab['t'] = tab['Marginal Effect'] / tab['s.e.']
tab.index.name = 'Var'
tab.round(6)

,Marginal Effect,s.e.,t
Var,,,
0,0.0,0.0,0.0


### Logit

In [21]:
b_lg = logit_tab.theta.values
me_black_lg = logit.G(x_me2@b_lg) - logit.G(x_me@b_lg)

In [22]:
#GAMMEL STD ERROR LØSNING

#gx0 = (np.exp(np.dot(x_me, b_lg))) / ((1 + np.exp(np.dot(x_me, b_lg)))**2)
#gx0 = (np.exp(np.dot(x_me2, b_lg))) / ((1 + np.exp(np.dot(x_me2, b_lg)))**2)

#grad_d_lg = gx2*x_me2 - gx0*x_me

#def get_se(grad, cov):
 #   cov_me = grad@cov@grad.T
 #   return np.sqrt(np.diag(cov_me))


#se_d_lg = get_se(grad_d_lg, logit_results['cov'])

In [23]:
def black_lg(b_lg):
    black_logit = logit.G(x_me2@b_lg) - logit.G(x_me@b_lg)
    return black_logit

qq = lambda b_lg: black_lg(b_lg)

grad_d_lg = est.centered_grad(qq,b_lg)

def get_se(grad, cov):
    cov_me = grad@cov@grad.T
    return np.sqrt(np.diag(cov_me))

se_d_lg = get_se(grad_d_lg, logit_results['cov'])

In [24]:
me_dict = {'Marginal Effect': me_black_lg[0],
           's.e.':            se_d_lg}
tab = pd.DataFrame(me_dict)
tab['t'] = tab['Marginal Effect'] / tab['s.e.']
tab.index.name = 'Var'
tab.round(6)

,Marginal Effect,s.e.,t
Var,,,
0,0.0,0.0,NaN


In [25]:
#p_values = 2 * t.sf(np.abs(me_black_pr[0]/se_d_pr), df=x.shape[0] - x.shape[1]).round(4)
#print(p_values)

[1.]
